# Balanced BSTs

## Agenda

1. Motivation
2. Defining "balance"
3. AVL trees
5. Other balanced trees

## 1. Motivation

In [ ]:
class BSTree:
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            
    def __init__(self):
        self.size = 0
        self.root = None
    
    def insert(self, val):
        assert val not in self
        def insert_rec(node):
            if not node:
                return BSTree.Node(val)
            elif val < node.val:
                node.left = insert_rec(node.left)
            else:
                node.right = insert_rec(node.right)
            return node
        self.root = insert_rec(self.root)
        self.size += 1

    def __contains__(self, val):
        def contains_rec(node):
            if not node:
                return False
            elif val == node.val:
                return True
            elif val < node.val:
                return contains_rec(node.left)
            else:
                return contains_rec(node.right)
        return contains_rec(self.root)        
                        
    def pprint(self, width=64):
        height = self.height()
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += '{val:^{width}}'.format(val='-', width=width//2**level)
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                repr_str += '{val:^{width}}'.format(val=n.val, width=width//2**level)
        print(repr_str)
    
    def height(self):
        def height_rec(t):
            if not t:
                return 0
            else:
                return max(1+height_rec(t.left), 1+height_rec(t.right))
        return height_rec(self.root)

In [ ]:

t = BSTree()
for x in range(6):
    t.insert(x)
t.pprint()

If a binary search tree is not balanced, it may degrade to a linked list! This results in $O(n)$ time complexity for all operations.

## 2. Defining "balance"

There are different criteria we can use to decide whether a binary tree is "balanced", e.g., 

- the *number* of nodes on either side of a given node
- the *height* of the subtrees on either side of a given node
- the *density* of nodes on either side of a given node
- the *shape* of the subtrees on either side of a given node (perhaps we only want to permit full or complete trees).

What are the pros and cons of some of these criteria?

### 2.1. Balance factor

*def.* in a height-balanced tree, the *balance factor* of a node is the height of its right subtree minus the height of its left subtree.

![](../images/16-bf-quiz.png)

To efficiently compute balance factors, we need to quickly access the height of each node in the tree. We can't afford to recompute the height of a subtree every time we need it!

In [ ]:
# Update the BSTree class so that:
# - the node class has a height attribute
# - the height of a node is updated during the `insert` operation
#   (do this without recalculating the height of sub-trees)

class BSTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            self.height = 1

    @staticmethod
    def update_height(n: Node):
        pass
            
    def insert(self, val):
        assert val not in self
        def insert_rec(node):
            if not node:
                return BSTree.Node(val)
            elif val < node.val:
                node.left = insert_rec(node.left)
                self.update_height(node)
            else:
                node.right = insert_rec(node.right)
                self.update_height(node)
            return node
        self.root = insert_rec(self.root)
        self.size += 1

    # updated pprint shows balance factors calculated from node heights
    def pprint(self, width=64):
        height = self.root.height if self.root else 0
        nodes  = [(self.root, 0)]
        prev_level = 0
        repr_str = ''
        while nodes:
            n,level = nodes.pop(0)
            if prev_level != level:
                prev_level = level
                repr_str += '\n'
            if not n:
                if level < height-1:
                    nodes.extend([(None, level+1), (None, level+1)])
                repr_str += f'{"-":^{width//2**level}}'
            elif n:
                if n.left or level < height-1:
                    nodes.append((n.left, level+1))
                if n.right or level < height-1:
                    nodes.append((n.right, level+1))
                bf = ((n.right.height if n.right else 0) 
                      - (n.left.height if n.left else 0))
                repr_str += f'{f"{n.val}[{bf}]":^{width//2**level}}'
        print(repr_str)

In [ ]:
import random
vals = list(range(10))
random.shuffle(vals)

t = BSTree()
for x in vals:
    t.insert(x)
t.pprint()

## 3. AVL (Adelson-Velsky and Landis) trees

*def.* an AVL tree is a height-balanced binary search tree where the balance factor is either -1, 0, or 1 for every node. We call this the *AVL property*.

An AVL tree is *self-balancing*. I.e., whenever the AVL property may be violated by an insertion/deletion, we must fix the tree before returning from the operation.

When may the AVL property be violated by insertions/deletions?

![](../images/16-avl-violations.png)

How can we fix these violations when they occur?

### 3.1. Essential operation: "rotation"

When the balance factor of a binary search tree node is $< -1$ (left-heavy) or $> 1$ (right heavy), we need an operation to redistribute the nodes in the tree to restore balance.

The operation needs to:
1. maintain the binary search tree property
2. predictably change the heights & balance factors of the nodes involved

The solution is a symmetric pair of operations called "rotations".

![](../images/16-rotations.png)

In [ ]:
# Implement in-place right rotation for the AVL tree node
# - be sure to update the heights of the nodes after rotation!

class AVLTree(BSTree):
    class Node:
        def __init__(self, val, left=None, right=None):
            self.val = val
            self.left = left
            self.right = right
            self.height = 1

        def rotate_right(self):
            pass
    
    def insert(self, val):
        assert val not in self
        def insert_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = insert_rec(node.left)
                self.update_height(node)
            else:
                node.right = insert_rec(node.right)
                self.update_height(node)
            return node
        self.root = insert_rec(self.root)
        self.size += 1

In [ ]:
t = AVLTree()
for x in range(6, 0, -1):
    t.insert(x)
t.pprint()

In [ ]:
t.root.rotate_right()
t.pprint()

In [ ]:
t.root.rotate_right()
t.pprint()

In [ ]:
t.root.left.rotate_right()
t.pprint()

Note that rotations do not *always* alter the height or balance factor of nodes in useful ways! 

![](../images/16-right-rotations.png)

### 3.2. "Out-of-balance" scenarios & rotation recipes

There are a finite number of ways a node can be out of balance. For each of these scenarios, there is a corresponding "recipe" that will restore balance. Convince yourself that these recipes work!

![](../images/16-rotation-recipes.png)

### 3.3. Insertion & Rebalancing

AVL tree insertion may cause an imbalance that requires precisely **one application of a rotation recipe** (i.e., 1-2 rotations) to fix.

In [ ]:
# Implement `rebalance` to deal with the "LL" case

class AVLTree(AVLTree):
    @staticmethod
    def rebalance(node):
        pass
            
    def insert(self, val):
        assert val not in self
        def insert_rec(node):
            if not node:
                return AVLTree.Node(val)
            elif val < node.val:
                node.left = insert_rec(node.left)
                self.update_height(node)
            else:
                node.right = insert_rec(node.right)
                self.update_height(node)

            # potentially rebalance
            self.rebalance(node)
            return node
        self.root = insert_rec(self.root)
        self.size += 1

In [ ]:
val = 50
t = AVLTree()

In [ ]:
# (evaluate multiple times with ctrl-enter)
t.insert(val)
val -= 1
t.pprint()

### 3.4. Deletion and Rebalancing

Deletions, unlike insertions, may cause an imbalance that requires **multiple applications of a rotation recipe** to fix! These imbalances, if they exist, will be found in ancestors of the deleted node.

![](../images/16-deletions.png)

### 3.5. AVL tree analysis

Central to the runtime complexity of AVL tree operations is the height of the tree $h$ in terms of the number of nodes $n$ in the tree.

Let us define $M(h)$ to be the *minimum number of nodes* in an AVL tree of height $h$. You can think of this as the most *sparsely-populated*, or *worst-case* AVL tree of height $h$.

It should be clear that $M(1) = 1$ and $M(2) = 2$. (Draw these trees!)

An AVL tree of height $=3$ with minimal nodes has one root node and subtrees of height $M(1)$ and $M(2)$, so $M(3) = 1 + M(1) + M(2)$.

Generally, we can say that $M(h) = 1 + M(h-1) + M(h-2)$. Can you see why?

Given this recursive formula, we can derive an upper bound on $h$ thusly:

$$
\begin{align*}
M(h) &= 1 + M(h-1) + M(h-2) \\
M(h-1) &= 1 + M(h-2) + M(h-3) \\
M(h) &= 1 + (1 + M(h-2) + M(h-3)) + M(h-2) & \text{by substitution}\\
&= 2 + 2M(h-2) + M(h-3) \\
M(h) &> 2M(h-2) \\
& > 4M(h-4) \\
& > 8M(h-6) \\
& > \ldots \\
& > 2^kM(h-2k) \\
& > 2^{\frac{h-1}{2}}M(1) & \text{letting $k = \frac{h-1}{2}$} \\
& > 2^{\frac{h-1}{2}} & \text{since $M(1) = 1$} \\
\log_2 M(h) &> \frac{h-1}{2} \\
h &< 2\log_2 M(h) + 1 \\
h &= O(\log M(h))
\end{align*}
$$

Since $M(h)$ is the minimum number of nodes in an AVL tree of height $h$, if $N$ is the number of nodes in *any AVL tree of height* $h$, $N \geq M(h)$, and therefore:

$$
h = O(\log N)
$$

This is a key conclusion! It means that *all operations* that have runtime proportional to the height of the AVL tree have runtime $= O(\log n)$.

## 4. Other optimized tree structures

AVL trees are not the only type of optimized tree data structure. Other examples include:

- Red-black trees
- Splay trees
- B-trees

**Red-black trees** are considered balanced so long as *the longest path from the root to a given leaf is no more than twice as long as the shortest path*. This is a weaker criterion than the AVL property, but it is cheaper to maintain and still guarantees $O(\log N)$ height for $N$ nodes.

**Splay trees** don't guarantee a specific height, but they continually reorganize the tree to ensure that the most frequently accessed nodes are closer to the root. This improves the *average-case* runtime complexity of operations on the tree.

**B-trees** are optimized for disk storage, where the cost of reading from disk is much higher than the cost of reading from memory. They are designed to minimize the number of disk reads required to find a node in the tree. Instead of using a binary tree structure, B-trees are *multiway trees*, where each node can have many children.